### Feature ablation study to investigate the impact of LIME and SAGE on the performance of the model

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
from numpy.core.numeric import full
import pickle
import time
import random 
import pprint
import warnings
from collections import Counter
from scipy.spatial import distance
from operator import itemgetter


from copy import deepcopy

from sklearn.metrics import accuracy_score, confusion_matrix, silhouette_score
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import linkage, fcluster

import skimage
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error

from skimage.color import gray2rgb, rgb2gray, label2rgb

from scipy.ndimage import convolve
from scipy.stats import zscore


# For keras dependencise
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Activation, Lambda, Reshape, Flatten
from keras.layers import BatchNormalization
from keras import losses
from keras import backend as K
from keras.losses import mse
from keras.optimizers import Adam

import sage


# For LRP Visuals
# from deepexplain.tensorflow import DeepExplain
from keras.models import Model

# LIME
import lime
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm
from skimage.segmentation import mark_boundaries, slic
from scipy import ndimage

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap


from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import fcluster

from PIL import Image


In [ ]:
model = load_model("NN4.h5")

# Load data
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

# Load the feature activations
X_train_act = np.load("X_train_act.npy")
X_test_act = np.load("X_test_act.npy")

# Load DeepLIFT contributions
X_train_cont = np.load("new_X_train_deeplift.npy")
X_test_cont = np.load("new_X_test_deeplift.npy")

# X_test_cont shape
X_test_cont.shape

In [ ]:
model.summary()

In [ ]:
from sklearn.metrics import accuracy_score

# Original model evaluation
original_predictions = model.predict(X_test)
original_predictions = np.argmax(original_predictions, axis=1)

original_accuracy = accuracy_score(y_test, original_predictions)
print(f"Original model accuracy: {original_accuracy}")


In [ ]:
# identify important features from pkl file
important_features_lime = pickle.load(open("top_1000_features_lime_fixed.pkl", "rb"))
important_features_aggregate = pickle.load(open("aggregated_result.pkl", "rb"))
important_features_sage = pickle.load(open("sage_importances.pkl", "rb"))


In [ ]:
# LIME ablation
X_test_lime = np.copy(X_test)

# Loop through each class to ablate the top features as per LIME
for class_label, features in important_features_lime.items():
    
    # Loop through the list of top features for that class
    for mask, image, score, imgid in features:
        # Apply the mask to all instances in the test set
        # Assuming the mask is of the same shape as each image in X_test
        X_test_lime[:, 0, :, :] *= (1 - mask)
        
# Make predictions with the ablated test set
ablated_predictions_lime = model.predict(X_test_lime)

# Calculate and print the overall accuracy
total_ablated_accuracy_lime = np.mean(np.argmax(ablated_predictions_lime, axis=1) == y_test)
print(f"Total accuracy after ablating top features from all classes: {total_ablated_accuracy_lime}")


In [ ]:
# aggregated superpixel ablation
X_test_aggregate = np.copy(X_test)

# Loop through each class to ablate the top features
for class_label, importances in important_features_aggregate.items():
    for mask, image, score, imgid in features:
        # Apply the mask to all instances in the test set
        # Assuming the mask is of the same shape as each image in X_test
        X_test_aggregate[:, 0, :, :] *= (1 - mask)
        
# Make predictions with the ablated test set
ablated_predictions_agg = model.predict(X_test_aggregate)

# Calculate and print the overall accuracy
# Assuming y_test is a 1D array of class labels
total_ablated_accuracy_aggregated = np.mean(np.argmax(ablated_predictions_agg, axis=1) == y_test)
print(f"Total accuracy after ablating top features from all classes: {total_ablated_accuracy_aggregated}")

In [ ]:
# sage ablation
X_test_sage = np.copy(X_test)

# Loop through each class to ablate the top features
for class_label, importances in important_features_sage.items():
    
    # Sort and get the top 3 features based on SAGE importance
    top_features = sorted(importances.items(), key=lambda x: x[1]['importance'], reverse=True)[:3]
    
    for index, feature_data in top_features:
        mask = feature_data['mask']
        
        # Apply the mask to all instances in the test set
        # This assumes that the mask and instances have compatible shapes
        X_test_sage[:, 0, :, :] *= (1 - mask)
        
# Make predictions using the ablated test set
ablated_predictions_sage = model.predict(X_test_sage)

# Calculate the overall accuracy
total_ablated_accuracy_sage = np.mean(np.argmax(ablated_predictions_sage, axis=1) == y_test)
print(f"Total accuracy after ablating top features from all classes: {total_ablated_accuracy_sage}")
